In [ ]:
df

In [ ]:
# test

import plotly.express as px
import pandas as pd

df1 = pd.read_excel(r"D:\GG\MINAS\map2.xlsx", sheet_name = "Sheet1")
df2 = pd.read_excel(r"D:\GG\MINAS\map2.xlsx", sheet_name = "Sheet2")
df = pd.merge(df1, df2, left_on = "UWI", right_on = "*WELL", how = "outer")

df.dropna(inplace=True)

df = df.groupby(["*WELL", "Latitude", "Longitude", "*YEAR", ], as_index=False)["*OIL"].sum()

In [ ]:
df

In [ ]:
type(geo_df["geometry"])

In [ ]:
import plotly.express as px
import geopandas as gpd

geo_df = gpd.read_file(gpd.datasets.get_path('naturalearth_cities'))

px.set_mapbox_access_token(open(".mapbox_token").read())
fig = px.scatter_geo(geo_df,
                    lat=geo_df.geometry.y,
                    lon=geo_df.geometry.x,
                    hover_name="name")
fig.show()

In [ ]:
from bubbly.bubbly import bubbleplot 

figure = bubbleplot(dataset=gapminder_indicators, x_column='gdpPercap', y_column='lifeExp', 
    bubble_column='country', time_column='year', size_column='pop', color_column='continent', 
    x_title="GDP per Capita", y_title="Life Expectancy", title='Gapminder Global Indicators',
    x_logscale=True, scale_bubble=3, height=650)

iplot(figure, config={'scrollzoom': True})

In [ ]:
import plotly.express as px

fig = px.scatter_geo(df,
                     lat="Latitude",
                     lon="Longitude",
                     hover_name="UWI")

fig.show()

In [ ]:
# test

import plotly.express as px
import pandas as pd

df = pd.read_excel(r"D:\GG\MINAS\map3.xlsx", sheet_name = "Geograph")
fig = px.scatter(df, x = "Longitude", y = "Latitude")
fig.show()

In [ ]:
import plotly.express as px
df = px.data.gapminder()
fig = px.scatter_geo(df, locations="iso_alpha", color="continent",
                     hover_name="country", size="pop",
                     animation_frame="year",
                     projection="natural earth")
fig.show()

# Calculating Average PERM and PORO for Specific Interval of Depth from .las Format 

In [ ]:
import pandas as pd
import numpy as np

wo = pd.read_excel(r"D:\GG\LQR MINAS\WO new.xlsx", sheet_name = 'Sheet2')

df_final = pd.DataFrame()

for j in range(0,120):

    f = open("D:\GG\LQR MINAS\LAS MINAS\\" + str(wo['UWI'][j]) + ".las", "r")
    contents = f.read()
    f.seek(0)
    content = f.readlines()

    if "SWJ" in contents:
        df = pd.DataFrame(content[63:])
    else:
        df = pd.DataFrame(content[62:])

    df.drop(0, axis=0, inplace=True)
    df[0] = df[0].str.replace('  ',' ')

    df['depth'] = np.nan
    df['perm'] = np.nan
    df['poro'] = np.nan
    df['well'] = np.nan

    for i in df.index:
            
        df['depth'][i] = df[0][i].split(' ')[1]

        if "SWJ" in contents:
            df['perm'][i] = df[0][i].split(' ')[17]
        else:
            df['perm'][i] = df[0][i].split(' ')[16]

        df['poro'][i] = df[0][i].split(' ')[3]

    df['depth'] = df['depth'].astype('float')
    df['poro'] = df['poro'].astype('float')
    df['perm'] = df['perm'].astype('float')
    df['well'] = str(wo['UWI'][j])

    f.close()

    df.drop(0, axis=1, inplace=True)
    df_new = df[(df['depth'] >= float(wo['TOP'][j])) & (df['depth'] <= float(wo['BOTTOM'][j]))]
    df_new = df_new.groupby('well', as_index=False)[['perm', 'poro']].mean()
    df_final = pd.concat([df_final, df_new])

df_final.to_excel("D:\GG\LQR MINAS\PERM & PORO.xlsx")

# Completion Update and Manipulation

In [47]:
import pandas as pd

df = pd.read_excel(r"D:\GG\LQR MINAS\Prod & Comp\Completion Final 2022.xlsx", sheet_name = "EV Merged until 2023")

In [28]:
for i,j in enumerate(df['STATUS'].unique()):
    df['STATUS'] = df['STATUS'].replace(i, j)

In [48]:
df['STATUS'] = df['STATUS'].replace('perforation', 1)
df['STATUS'] = df['STATUS'].replace('squeeze', 2)
df['STATUS'] = df['STATUS'].replace('plug', 3)

In [49]:
df = df.sort_values(by=['WELL', 'STATUS', 'DATE', 'TOP'])

In [50]:
df['STATUS'] = df['STATUS'].replace(1, 'perforation')
df['STATUS'] = df['STATUS'].replace(2, 'squeeze')
df['STATUS'] = df['STATUS'].replace(3, 'plug')

In [51]:
df.to_excel(r"D:\GG\LQR MINAS\Prod & Comp\New Perfo 2023.xlsx", index = False)

In [57]:
import xlsxwriter

wb = xlsxwriter.Workbook(r"D:\GG\LQR MINAS\Prod & Comp\New Perfo EV 2023.xlsx", )

sheet = wb.add_worksheet("EV")

row_count = 2
df = pd.read_excel(r"D:\GG\LQR MINAS\Prod & Comp\New Perfo 2023.xlsx", sheet_name = "Ready")

sheet.write(0,0, 'UNITS FIELD')

for i in df.index:
    
    sheet.write(row_count, 0, str(df['COMP'][i]))

    if i == 0:
        sheet.write(row_count, 0, "WELLNAME " + "\"" + str(df['WELL'][i]) + "\"")

    if str(df['WELL'][i]) == '1':
        break

    if str(df['WELL'][i]) == str(df['WELL'][i+1]):
        
        row_count += 1

        if 'perforation' in str(df['COMP'][i]) and 'squeeze' in str(df['COMP'][i+1]):
            row_count += 1
        elif 'squeeze' in str(df['COMP'][i]) and 'plug' in str(df['COMP'][i+1]):
            row_count += 1
        else: 
            pass

    elif str(df['WELL'][i]) != str(df['WELL'][i+1]):
        row_count+=2
        sheet.write(row_count, 0, "WELLNAME " + "\"" + str(df['WELL'][i+1]) + "\"")
        row_count+=1

wb.close()